In [1]:
import pandas as pd

# Загрузка данных из Excel-файла
df = pd.read_excel('data/data.xlsx')

# Проверка типов данных
print("Типы данных до очистки:")
print(df.dtypes)

# Функция для преобразования строк в числа с обработкой ошибок
def convert_to_numeric(column):
    # Пытаемся преобразовать столбец в числовой формат
    converted = pd.to_numeric(column, errors='coerce')
    # Находим строки, которые не удалось преобразовать
    invalid_rows = column[converted.isna()]
    if not invalid_rows.empty:
        print(f"Найдены некорректные значения в столбце {column.name}:")
        print(invalid_rows)
    return converted

# Применяем функцию к столбцам Quantity и Product Price
df['Quantity'] = convert_to_numeric(df['Quantity'])
df['Product Price'] = convert_to_numeric(df['Product Price'])

# Удаляем строки с некорректными значениями (если они есть)
df = df.dropna(subset=['Quantity', 'Product Price'])

# Удаление дубликатов
df = df.drop_duplicates()

# Заполнение пропущенных значений (если есть другие столбцы с пропусками)
df = df.fillna(0)

# Проверка данных после очистки
print("\nТипы данных после очистки:")
print(df.dtypes)
print("\nПервые 5 строк данных после очистки:")
print(df.head())

Типы данных до очистки:
Order Number               int64
Order Date        datetime64[ns]
Item Name                 object
Quantity                 float64
Product Price            float64
Total products             int64
Столбец1                 float64
dtype: object

Типы данных после очистки:
Order Number               int64
Order Date        datetime64[ns]
Item Name                 object
Quantity                 float64
Product Price            float64
Total products             int64
Столбец1                 float64
dtype: object

Первые 5 строк данных после очистки:
   Order Number          Order Date   Item Name  Quantity  Product Price  \
0         12730 2018-12-30 22:17:00  Aloo Chaat       1.0           4.95   
1         12672 2018-12-26 18:13:00  Aloo Chaat       1.0           4.95   
2         12652 2018-12-24 18:46:00  Aloo Chaat       1.0           4.95   
3         12588 2018-12-21 21:14:00  Aloo Chaat       1.0           4.95   
4         12511 2018-12-15 19:56:00  Alo

In [2]:
# Расчет базовых статистических показателей
stats = {
    'Кол-во заказов': df['Order Number'].nunique(),
    'Среднее количество блюд в заказе': df['Total products'].mean(),
    'Медиана количества блюд в заказе': df['Total products'].median(),
    'Мода количества блюд в заказе': df['Total products'].mode()[0],
    'Дисперсия количества блюд в заказе': df['Total products'].var(),
    'Стандартное отклонение количества блюд в заказе': df['Total products'].std()
}

# Вывод статистических показателей
print("\nБазовые статистические показатели:")
for key, value in stats.items():
    print(f'{key}: {value}')

# Параметры транспорта
bike_capacity = 10  # Максимальное число блюд в одном заказе для велосипеда
bike_orders = 4     # Максимальное число заказов на один рейс для велосипеда
car_capacity = 16   # Максимальное число блюд в одном заказе для автомобиля
car_orders = 10     # Максимальное число заказов на один рейс для автомобиля

# Расчет доли клиентов, которые смогут оформить заказ
bike_able = df[df['Total products'] <= bike_capacity].shape[0] / df.shape[0]
car_able = df[df['Total products'] <= car_capacity].shape[0] / df.shape[0]

# Расчет доли клиентов, которые не смогут оформить заказ
bike_unable = 1 - bike_able
car_unable = 1 - car_able

# Расчет потерянной выручки
bike_lost_revenue = df[df['Total products'] > bike_capacity]['Product Price'].sum()
car_lost_revenue = df[df['Total products'] > car_capacity]['Product Price'].sum()

# Расчет доли потерянной выручки
total_revenue = df['Product Price'].sum()
bike_lost_revenue_share = bike_lost_revenue / total_revenue
car_lost_revenue_share = car_lost_revenue / total_revenue

# Расчет необходимого количества транспорта
bike_needed = df[df['Total products'] <= bike_capacity].shape[0] // bike_orders
car_needed = df[df['Total products'] <= car_capacity].shape[0] // car_orders

# Расчет расходов на закупку транспорта
bike_cost = 800
car_cost = 9000

bike_total_cost = bike_needed * bike_cost
car_total_cost = car_needed * car_cost

# Вывод результатов
print("\nМетрики для транспорта:")
print(f'Доля клиентов, которые смогут оформить заказ (велосипеды): {bike_able:.2%}')
print(f'Доля клиентов, которые смогут оформить заказ (автомобили): {car_able:.2%}')
print(f'Доля клиентов, которые не смогут оформить заказ (велосипеды): {bike_unable:.2%}')
print(f'Доля клиентов, которые не смогут оформить заказ (автомобили): {car_unable:.2%}')
print(f'Потерянная выручка (велосипеды): {bike_lost_revenue:.2f}')
print(f'Потерянная выручка (автомобили): {car_lost_revenue:.2f}')
print(f'Доля потерянной выручки (велосипеды): {bike_lost_revenue_share:.2%}')
print(f'Доля потерянной выручки (автомобили): {car_lost_revenue_share:.2%}')
print(f'Необходимо велосипедов: {bike_needed}')
print(f'Необходимо автомобилей: {car_needed}')
print(f'Расходы на закупку велосипедов: {bike_total_cost:.2f}')
print(f'Расходы на закупку автомобилей: {car_total_cost:.2f}')


Базовые статистические показатели:
Кол-во заказов: 4560
Среднее количество блюд в заказе: 7.147744390708
Медиана количества блюд в заказе: 6.0
Мода количества блюд в заказе: 5
Дисперсия количества блюд в заказе: 28.0259407990358
Стандартное отклонение количества блюд в заказе: 5.293953229774116

Метрики для транспорта:
Доля клиентов, которые смогут оформить заказ (велосипеды): 88.66%
Доля клиентов, которые смогут оформить заказ (автомобили): 97.91%
Доля клиентов, которые не смогут оформить заказ (велосипеды): 11.34%
Доля клиентов, которые не смогут оформить заказ (автомобили): 2.09%
Потерянная выручка (велосипеды): 14294.50
Потерянная выручка (автомобили): 3336.10
Доля потерянной выручки (велосипеды): 10.66%
Доля потерянной выручки (автомобили): 2.49%
Необходимо велосипедов: 5591
Необходимо автомобилей: 2469
Расходы на закупку велосипедов: 4472800.00
Расходы на закупку автомобилей: 22221000.00


In [3]:
# Аналитическая записка
analytical_note = """

Цель анализа: Определить оптимальный тип транспорта для доставки заказов в кафе индийской кухни.

Данные:
- Время заказов за год.
- Заказанные блюда и их количество.
- Цена каждого блюда.
- Количество блюд в заказе.

Результаты анализа:
1. Базовые статистические показатели:
   - Среднее количество блюд в заказе: {mean:.2f}
   - Медиана количества блюд в заказе: {median}
   - Мода количества блюд в заказе: {mode}
   - Дисперсия: {variance:.2f}
   - Стандартное отклонение: {std_dev:.2f}

2. Метрики для транспорта:
   - Велосипеды:
     - Доля клиентов, которые смогут оформить заказ: {bike_able:.2%}
     - Доля клиентов, которые не смогут оформить заказ: {bike_unable:.2%}
     - Потерянная выручка: {bike_lost_revenue:.2f}
     - Доля потерянной выручки: {bike_lost_revenue_share:.2%}
     - Необходимое количество велосипедов: {bike_needed}
     - Расходы на закупку велосипедов: {bike_total_cost:.2f}

   - Автомобили:
     - Доля клиентов, которые смогут оформить заказ: {car_able:.2%}
     - Доля клиентов, которые не смогут оформить заказ: {car_unable:.2%}
     - Потерянная выручка: {car_lost_revenue:.2f}
     - Доля потерянной выручки: {car_lost_revenue_share:.2%}
     - Необходимое количество автомобилей: {car_needed}
     - Расходы на закупку автомобилей: {car_total_cost:.2f}

Рекомендации:
- Если приоритетом является минимизация потери клиентов и выручки, рекомендуется выбрать автомобили.
- Если приоритетом является минимизация затрат на закупку транспорта, можно рассмотреть велосипеды, но с учетом потери части клиентов и выручки.

Итоговый выбор: Рекомендуется выбрать автомобили для доставки заказов.
""".format(
    mean=stats['Среднее количество блюд в заказе'],
    median=stats['Медиана количества блюд в заказе'],
    mode=stats['Мода количества блюд в заказе'],
    variance=stats['Дисперсия количества блюд в заказе'],
    std_dev=stats['Стандартное отклонение количества блюд в заказе'],
    bike_able=bike_able,
    car_able=car_able,
    bike_unable=bike_unable,
    car_unable=car_unable,
    bike_lost_revenue=bike_lost_revenue,
    car_lost_revenue=car_lost_revenue,
    bike_lost_revenue_share=bike_lost_revenue_share,
    car_lost_revenue_share=car_lost_revenue_share,
    bike_needed=bike_needed,
    car_needed=car_needed,
    bike_total_cost=bike_total_cost,
    car_total_cost=car_total_cost
)

# Вывод аналитической записки
print("\nАналитическая записка:")
print(analytical_note)


Аналитическая записка:


Цель анализа: Определить оптимальный тип транспорта для доставки заказов в кафе индийской кухни.

Данные:
- Время заказов за год.
- Заказанные блюда и их количество.
- Цена каждого блюда.
- Количество блюд в заказе.

Результаты анализа:
1. Базовые статистические показатели:
   - Среднее количество блюд в заказе: 7.15
   - Медиана количества блюд в заказе: 6.0
   - Мода количества блюд в заказе: 5
   - Дисперсия: 28.03
   - Стандартное отклонение: 5.29

2. Метрики для транспорта:
   - Велосипеды:
     - Доля клиентов, которые смогут оформить заказ: 88.66%
     - Доля клиентов, которые не смогут оформить заказ: 11.34%
     - Потерянная выручка: 14294.50
     - Доля потерянной выручки: 10.66%
     - Необходимое количество велосипедов: 5591
     - Расходы на закупку велосипедов: 4472800.00

   - Автомобили:
     - Доля клиентов, которые смогут оформить заказ: 97.91%
     - Доля клиентов, которые не смогут оформить заказ: 2.09%
     - Потерянная выручка: 3336.10
  